In [7]:
import pandas as pd
import numpy as np
import cdsapi
import utm
import os 
import json
import sys

In [8]:
df = pd.read_excel('ตารางการสำสวจโรคยางพารา.xlsx',sheet_name='สตูล66')

In [9]:
df.head()

,ลำดับ,ชื่อเกษตรกร,วันที่สำรวจ,ตำบล,อำเภอ,จังหวัด,พิกัด,Unnamed: 7,อายุต้นยาง(ปี),ชนิดดิน,พันธุ์ยางพารา,โรคที่พบ,เปอร์เซนต์การเกิดโรค,ระดับความรุนแรง
0,NaN,NaN,NaN,NaN,NaN,NaN,x,y,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,15/12/66,ห้วยน้ำดำใน,ควนกาหลง,สตูล,616259,758828,-,-,-,โรใบร่วงชนิดใหม่,51-75%,4.0
2,2.0,NaN,15/12/66,ห้วยน้ำดำใน,ควนกาหลง,สตูล,615268,757701,-,-,-,โรใบร่วงชนิดใหม่,51-75%,4.0
3,3.0,NaN,15/12/66,ห้วยน้ำดำใน,ควนกาหลง,สตูล,615218,757721,-,-,-,โรใบร่วงชนิดใหม่,25-50%,3.0
4,4.0,NaN,15/12/66,ห้วยน้ำดำใน,ควนกาหลง,สตูล,616289,758009,-,-,-,โรใบร่วงชนิดใหม่,51-75%,4.0


In [10]:
df = df.iloc[1:,[2,3,4,5,6,7]]

In [11]:
df

,วันที่สำรวจ,ตำบล,อำเภอ,จังหวัด,พิกัด,Unnamed: 7
1,15/12/66,ห้วยน้ำดำใน,ควนกาหลง,สตูล,616259,758828
2,15/12/66,ห้วยน้ำดำใน,ควนกาหลง,สตูล,615268,757701
3,15/12/66,ห้วยน้ำดำใน,ควนกาหลง,สตูล,615218,757721
4,15/12/66,ห้วยน้ำดำใน,ควนกาหลง,สตูล,616289,758009
5,15/12/66,บ้านควน,เมือง,สตูล,615633,738138
6,15/12/66,บ้านควน,เมือง,สตูล,615587,738025
7,15/12/66,บ้านควน,เมือง,สตูล,615062,740497
8,15/12/66,บ้านควน,เมือง,สตูล,614930,740444


In [13]:
df = df[:].rename({'วันที่สำรวจ':'date','พิกัด':'x','Unnamed: 7':'y'},axis=1)
def adjust_year_to_gregorian(date_str):
    date_obj = pd.to_datetime(date_str, format='%d/%m/%y')
    gregorian_year = date_obj.year - 43
    # Recreate the date with the adjusted year
    return date_obj.replace(year=gregorian_year).strftime('%Y-%m-%d')

df['date'] = df['date'].apply(adjust_year_to_gregorian)


In [14]:
df

,date,ตำบล,อำเภอ,จังหวัด,x,y
1,2023-12-15,ห้วยน้ำดำใน,ควนกาหลง,สตูล,616259,758828
2,2023-12-15,ห้วยน้ำดำใน,ควนกาหลง,สตูล,615268,757701
3,2023-12-15,ห้วยน้ำดำใน,ควนกาหลง,สตูล,615218,757721
4,2023-12-15,ห้วยน้ำดำใน,ควนกาหลง,สตูล,616289,758009
5,2023-12-15,บ้านควน,เมือง,สตูล,615633,738138
6,2023-12-15,บ้านควน,เมือง,สตูล,615587,738025
7,2023-12-15,บ้านควน,เมือง,สตูล,615062,740497
8,2023-12-15,บ้านควน,เมือง,สตูล,614930,740444


In [16]:
def utmToLatLon(x:list, y:list, zone:int = 47) -> tuple:
    x, y = np.mean(x), np.mean(y) #find mid point
    return utm.to_latlon(x,y, zone_number=zone, zone_letter='N') #return (lat, lon)

In [17]:
# for i , data in df.iterrows():
#     lat, lon = utmToLatLon(x=[data['x']], y=[data['y']], zone=47)
#     year = int(data['date'].split('-')[0])
#     month = int(data['date'].split('-')[1])
#     day = int(data['date'].split('-')[2])
#     print(f"\nDownloading {data['date']}-{data['ตำบล']}-{data['อำเภอ']}-{data['จังหวัด']} ({lat}, {lon})")
   

In [9]:
DIR_OUTPUT = f'../Satun-Phang-Nga/1nc/'
os.makedirs(DIR_OUTPUT, exist_ok=True)
c = cdsapi.Client()
for i , data in df.iterrows():
    lat, lon = utmToLatLon(x=[data['x']], y=[data['y']], zone=47)
    year = int(data['date'].split('-')[0])
    month = int(data['date'].split('-')[1])
    day = int(data['date'].split('-')[2])
    print(f"\nDownloading {data['date']}-{data['ตำบล']}-{data['อำเภอ']}-{data['จังหวัด']} ({lat}, {lon})")
    c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': [
                    '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
                    '2m_temperature', 'downward_uv_radiation_at_the_surface', 'evaporation',
                    'soil_temperature_level_1', 'surface_pressure', 'total_precipitation',
                ],
                'year': year,
                'month': [str(i) for i in range(month-1, month+1)],
                'day': [str(i) for i in range(1, 32)],
                'area' : [lat, lon, lat, lon],                  
                'time': [
                    '00:00', '06:00', '12:00',
                    '18:00',
                ],
            },
            f"{DIR_OUTPUT}{i}-{data['date']}-{data['ตำบล']}-{data['อำเภอ']}-{data['จังหวัด']}.nc")

2024-07-19 15:37:19,966 INFO Welcome to the CDS
2024-07-19 15:37:19,967 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 15:37:20,302 INFO Request is queued
2024-07-19 15:40:14,721 INFO Request is running
2024-07-19 15:41:42,813 INFO Request is completed
2024-07-19 15:41:42,814 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data6/adaptor.mars.internal-1721378477.470082-12222-14-3f654b21-e0f7-436e-81cc-35d21b3365f9.nc to ../Satun-Phang-Nga/1nc/1-2023-12-15-ห้วยน้ำดำใน-ควนกาหลง-สตูล.nc (8.4K)
2024-07-19 15:41:45,559 INFO Download rate 3.1K/s


2024-07-19 15:41:46,029 INFO Welcome to the CDS
2024-07-19 15:41:46,030 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 15:41:46,317 INFO Request is queued
2024-07-19 15:42:08,542 INFO Request is running
2024-07-19 15:44:39,954 INFO Request is completed
2024-07-19 15:44:39,955 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data1/adaptor.mars.internal-1721378644.2688804-8634-20-765c50e2-b8a2-4a16-af0d-6dec621d013a.nc to ../Satun-Phang-Nga/1nc/2-2023-12-15-ห้วยน้ำดำใน-ควนกาหลง-สตูล.nc (8.4K)
2024-07-19 15:44:42,577 INFO Download rate 3.2K/s


2024-07-19 15:44:43,055 INFO Welcome to the CDS
2024-07-19 15:44:43,057 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 15:44:43,312 INFO Request is queued
2024-07-19 15:44:57,677 INFO Request is running
2024-07-19 15:47:36,915 INFO Request is completed
2024-07-19 15:47:36,916 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data9/adaptor.mars.internal-1721378808.557851-32090-14-effb1ba0-bbba-4428-92f9-70160c23ca9b.nc to ../Satun-Phang-Nga/1nc/3-2023-12-15-ห้วยน้ำดำใน-ควนกาหลง-สตูล.nc (8.4K)
2024-07-19 15:47:39,673 INFO Download rate 3K/s


2024-07-19 15:47:40,195 INFO Welcome to the CDS
2024-07-19 15:47:40,197 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 15:47:40,456 INFO Request is queued
2024-07-19 15:47:46,079 INFO Request is running
2024-07-19 15:50:34,233 INFO Request is completed
2024-07-19 15:50:34,235 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data3/adaptor.mars.internal-1721378973.4881623-23375-7-40681a1f-2f58-4825-bbee-c4576d1281e7.nc to ../Satun-Phang-Nga/1nc/4-2023-12-15-ห้วยน้ำดำใน-ควนกาหลง-สตูล.nc (8.4K)
2024-07-19 15:50:36,849 INFO Download rate 3.2K/s


2024-07-19 15:50:37,334 INFO Welcome to the CDS
2024-07-19 15:50:37,335 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 15:50:37,682 INFO Request is queued
2024-07-19 15:51:28,860 INFO Request is running
2024-07-19 15:54:59,317 INFO Request is completed
2024-07-19 15:54:59,318 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data3/adaptor.mars.internal-1721379198.220036-23375-10-ee874b4d-6c6d-4582-884e-0b96b89ad95a.nc to ../Satun-Phang-Nga/1nc/5-2023-12-15-บ้านควน-เมือง-สตูล.nc (8.4K)
2024-07-19 15:55:01,000 INFO Download rate 5K/s


2024-07-19 15:55:01,495 INFO Welcome to the CDS
2024-07-19 15:55:01,496 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 15:55:01,750 INFO Request is queued
2024-07-19 15:55:04,719 INFO Request is running
2024-07-19 15:57:55,376 INFO Request is completed
2024-07-19 15:57:55,378 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data9/adaptor.mars.internal-1721379413.2963154-32371-9-104ff5e4-7353-40fb-b8cc-f4ef6ac69848.nc to ../Satun-Phang-Nga/1nc/6-2023-12-15-บ้านควน-เมือง-สตูล.nc (8.4K)
2024-07-19 15:57:58,106 INFO Download rate 3.1K/s


2024-07-19 15:57:58,579 INFO Welcome to the CDS
2024-07-19 15:57:58,579 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 15:57:58,851 INFO Request is queued
2024-07-19 15:58:00,082 INFO Request is running
2024-07-19 16:00:52,461 INFO Request is completed
2024-07-19 16:00:52,462 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data8/adaptor.mars.internal-1721379590.8515186-30539-5-d89b07a0-6dff-49f1-ba05-96612617cb4a.nc to ../Satun-Phang-Nga/1nc/7-2023-12-15-บ้านควน-เมือง-สตูล.nc (8.4K)
2024-07-19 16:00:53,944 INFO Download rate 5.7K/s


2024-07-19 16:00:54,407 INFO Welcome to the CDS
2024-07-19 16:00:54,409 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-19 16:00:54,685 INFO Request is queued
2024-07-19 16:00:57,660 INFO Request is running
2024-07-19 16:05:16,074 INFO Request is completed
2024-07-19 16:05:16,074 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data9/adaptor.mars.internal-1721379855.173554-30539-7-037f1efb-e61b-4317-8f68-f92280cfdb0c.nc to ../Satun-Phang-Nga/1nc/8-2023-12-15-บ้านควน-เมือง-สตูล.nc (8.4K)
2024-07-19 16:05:17,669 INFO Download rate 5.3K/s
